# SQL Queries
After uploading the 2 raw datasets - station.csv and resultphyschem.csv to the SQL database, the following queries were carried out to extract important columns and rows.

```sql
-- Altering column data types from the data importing process
ALTER TABLE stations
	ALTER COLUMN latitudemeasure TYPE DOUBLE PRECISION USING latitudemeasure::DOUBLE PRECISION,
	ALTER COLUMN longitudemeasure TYPE DOUBLE PRECISION USING longitudemeasure::DOUBLE PRECISION,
	ALTER COLUMN drainageareameasure_measurevalue TYPE DOUBLE PRECISION USING drainageareameasure_measurevalue::DOUBLE PRECISION,
	ALTER COLUMN contributingdrainageareameasure_measurevalue TYPE DOUBLE PRECISION USING contributingdrainageareameasure_measurevalue::DOUBLE PRECISION,
	ALTER COLUMN verticalaccuracymeasure_measurevalue TYPE DOUBLE PRECISION USING verticalaccuracymeasure_measurevalue::DOUBLE PRECISION,
	ALTER COLUMN verticalmeasure_measurevalue TYPE DOUBLE PRECISION USING verticalmeasure_measurevalue::DOUBLE PRECISION;

ALTER TABLE results
	ALTER COLUMN activitystartdate TYPE DATE USING activitystartdate::DATE,
	ALTER COLUMN activitystarttime_time TYPE TIME USING activitystarttime_time::TIME,
	ALTER COLUMN activityenddate TYPE DATE USING activityenddate::DATE,
	ALTER COLUMN activityendtime_time TYPE TIME USING activityendtime_time::TIME,
	ALTER COLUMN activitylocation_latitudemeasure TYPE DOUBLE PRECISION USING activitylocation_latitudemeasure::DOUBLE PRECISION,
	ALTER COLUMN activitylocation_longitudemeasure TYPE DOUBLE PRECISION USING activitylocation_longitudemeasure::DOUBLE PRECISION;

-- Creating a column to convert numerical results values to double precision
ALTER TABLE results ADD COLUMN result_numeric DOUBLE PRECISION;
-- Creating a column to convert numerical detection limit values to double precision
ALTER TABLE results ADD COLUMN limit_numeric DOUBLE PRECISION;

-- Converting all numerical entries to double precision for detection limits, nulling all the others
UPDATE results
SET limit_numeric = CASE
	WHEN detectionquantitationlimitmeasure_measurevalue ~ '^[0-9.]+$'
		THEN CAST(detectionquantitationlimitmeasure_measurevalue AS DOUBLE PRECISION)
	ELSE NULL
END;

-- Converting all numerical entries to double precision for result values as well as text entries which imply low or zero values for calculations and analysis as well as zeroing anything else outside of interest
UPDATE results
SET result_numeric = CASE
	WHEN resultmeasurevalue ~ '^[0-9.]+$'
		THEN CAST(resultmeasurevalue AS DOUBLE PRECISION)
		
	WHEN resultmeasurevalue LIKE '<%'
		THEN CAST(REPLACE(resultmeasurevalue, '<', '') AS DOUBLE PRECISION) / 2
		
	WHEN resultdetectionconditiontext IN ('ND', 'Not Detected', 'Detected Not Quantified', 'Not Detected at Reporting Limit', 'Below Detection Limit', 'Below Reporting Limit', 'Present Below Quantification Limit') OR resultmeasurevalue IN ('ND', 'Not Detected')
		THEN limit_numeric / 2

	ELSE NULL
END;
```

The final query was ran to address the entries where result values were either missing or reported as something along the lines of "Not Detected". The query searches for that specific entry's detection limit, halves it and returns that calculation as the result value. This was done to address as many missing values to not skew the level of conformity as much.

```sql
-- Joining the results and stations tables
CREATE VIEW v_water_quality_analysis AS
SELECT
	s.monitoringlocationidentifier,
	s.monitoringlocationname,
	s.latitudemeasure,
	s.longitudemeasure,
	r.activitystartdate,
	r.characteristicname,
	r.result_numeric AS result_value,
	r.resultmeasure_measureunitcode AS units
FROM results AS r
JOIN stations AS s ON r.monitoringlocationidentifier = s.monitoringlocationidentifier
WHERE r.result_numeric IS NOT NULL;
```

This query merges important columns from the results and stations tables to remove as much of the clutter of the metadata from both as possible.

```sql
CREATE TABLE health_standards (
	characteristicname TEXT,
	mcl_value DOUBLE PRECISION,
	units TEXT,
	description TEXT
);

INSERT INTO health_standards VALUES
('Arsenic', 0.010, 'mg/l', 'Human Health - Drinking Water'),
('Lead', 0.015, 'mg/l', 'Action Level for Infrastructure'),
('Nitrate', 10.0, 'mg/l', 'Common Agricultural Runoff'),
('Mercury', 0.002, 'mg/l', 'Bioaccumulation/Fish Health'),
('Copper', 1.3, 'mg/l', 'Industrial/Plumbing'),
('Uranium', 30.0, 'ug/l', 'Common in CA Ground Water'),
('Fluoride', 4.0, 'mg/l', 'Natural Occurence'),
('Turbidity', 5.0, 'NTU', 'Storm Runoff/Clarity'),
('Total Dissolved Solids', 500.0, 'mg/l', 'Seconday Standard - Taste'),
('Selenium', 0.05, 'mg/l', 'Agricultural Runoff');
```

This query creates a new table to hold the information about each contaminant''s EPA limits 

```sql
-- Joins the v_water_quality_analysis and health_standards tables to create the master file to be used for analysis
SELECT
	v.*,
	h.mcl_value,
	h.units as mcl_units
FROM v_water_quality_analysis AS v
JOIN health_standards AS h ON v.characteristicname = h.characteristicname;
```

This query creates the 'water_quality_master_file' table which will be used for the excel data cleaning and validation, then the Python analysis and summarization. 

# Python Analysis and Summarization

In [60]:
# Loading the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
# Importing the CSV as a dataframe to use
water_quality = pd.read_csv("water_quality_master_file.csv")

water_quality

,monitoringlocationidentifier,monitoringlocationname,latitudemeasure,longitudemeasure,activitystartdate,characteristicname,result_value,units,mcl_value,mcl_units,result_mg_l,exceedance_flag,is_proxy_unit
0,GLEC-10124,Colorado River,33.258610,-114.672240,11/11/2023,Arsenic,0.0027,mg/L,0.01,mg/l,0.0027,0,NaN
1,SOBOBA_WQX-IC2,Indian Creek 2nd Crossing,33.463970,-116.509600,16/11/2023,Arsenic,4.1000,ug/L,0.01,mg/l,0.0041,0,NaN
2,GLEC-10124,Colorado River,33.258610,-114.672240,14/09/2023,Arsenic,0.0031,mg/L,0.01,mg/l,0.0031,0,NaN
3,USGS-10260950,WF MOJAVE R AB MOJAVE R FORKS RES NR HESPERIA CA,34.338892,-117.257821,26/10/2023,Arsenic,1.9000,ug/L,0.01,mg/l,0.0019,0,NaN
4,USGS-11074000,SANTA ANA R BL PRADO DAM CA,33.883349,-117.645330,30/10/2023,Arsenic,3.0000,ug/L,0.01,mg/l,0.0030,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5291,MORONGO1_WQX-M003,Millard 003,33.961460,-116.800000,19/04/2023,Selenium,2.5000,ug/L,0.05,mg/l,0.0025,0,NaN
5292,PPNPROGRAM-SS7,Pond #1,39.550000,-123.438400,12/01/2023,Selenium,0.0100,mg/L,0.05,mg/l,0.0100,0,NaN
5293,GLEC-10124,Colorado River,33.258610,-114.672240,14/09/2023,Selenium,0.0015,mg/L,0.05,mg/l,0.0015,0,NaN
5294,21NEV1_WQX-C8,West Fork Carson River @ Paynesville,38.808781,-119.777199,05/12/2023,Selenium,2.5000,ug/L,0.05,mg/l,0.0025,0,NaN


In [62]:
# Dropping rows that are missing station names
water_quality = water_quality.dropna(subset="monitoringlocationname")

In [63]:
water_quality

,monitoringlocationidentifier,monitoringlocationname,latitudemeasure,longitudemeasure,activitystartdate,characteristicname,result_value,units,mcl_value,mcl_units,result_mg_l,exceedance_flag,is_proxy_unit
0,GLEC-10124,Colorado River,33.258610,-114.672240,11/11/2023,Arsenic,0.0027,mg/L,0.01,mg/l,0.0027,0,NaN
1,SOBOBA_WQX-IC2,Indian Creek 2nd Crossing,33.463970,-116.509600,16/11/2023,Arsenic,4.1000,ug/L,0.01,mg/l,0.0041,0,NaN
2,GLEC-10124,Colorado River,33.258610,-114.672240,14/09/2023,Arsenic,0.0031,mg/L,0.01,mg/l,0.0031,0,NaN
3,USGS-10260950,WF MOJAVE R AB MOJAVE R FORKS RES NR HESPERIA CA,34.338892,-117.257821,26/10/2023,Arsenic,1.9000,ug/L,0.01,mg/l,0.0019,0,NaN
4,USGS-11074000,SANTA ANA R BL PRADO DAM CA,33.883349,-117.645330,30/10/2023,Arsenic,3.0000,ug/L,0.01,mg/l,0.0030,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5291,MORONGO1_WQX-M003,Millard 003,33.961460,-116.800000,19/04/2023,Selenium,2.5000,ug/L,0.05,mg/l,0.0025,0,NaN
5292,PPNPROGRAM-SS7,Pond #1,39.550000,-123.438400,12/01/2023,Selenium,0.0100,mg/L,0.05,mg/l,0.0100,0,NaN
5293,GLEC-10124,Colorado River,33.258610,-114.672240,14/09/2023,Selenium,0.0015,mg/L,0.05,mg/l,0.0015,0,NaN
5294,21NEV1_WQX-C8,West Fork Carson River @ Paynesville,38.808781,-119.777199,05/12/2023,Selenium,2.5000,ug/L,0.05,mg/l,0.0025,0,NaN


In [64]:
# Creating a summary table which groups each station by its total number of tests for each contaminant and how many times they failed to conform to MCLs
station_summary = water_quality.groupby(["monitoringlocationidentifier", "characteristicname"]).agg(
    total_samples = ("result_mg_l", "count"),
    compliance_failures = ("exceedance_flag", "sum"),
    peak_result = ("result_mg_l", "max"),
    mcl_value = ("mcl_value", "first")
).reset_index()

station_summary

,monitoringlocationidentifier,characteristicname,total_samples,compliance_failures,peak_result,mcl_value
0,11NPSWRD_WQX-SFAN_I&M_BG2,Nitrate,13,0,1.50,10.0
1,11NPSWRD_WQX-SFAN_I&M_BG2,Turbidity,13,2,11.30,5.0
2,11NPSWRD_WQX-SFAN_I&M_BJC1,Nitrate,13,0,0.41,10.0
3,11NPSWRD_WQX-SFAN_I&M_BJC1,Turbidity,13,0,4.34,5.0
4,11NPSWRD_WQX-SFAN_I&M_CDC2,Nitrate,5,0,0.54,10.0
...,...,...,...,...,...,...
803,USGS-391448122565601,Turbidity,7,6,520.00,5.0
804,YUROKTEP_WQX-TC,Turbidity,16,8,27.56,5.0
805,YUROKTEP_WQX-TG,Turbidity,19,8,36.00,5.0
806,YUROKTEP_WQX-TR,Turbidity,24,6,43.10,5.0


In [65]:
# Creating columns for exceedance rate (proportion of times samples failed compliance) and severity ratio (how many times the MCL did the peak sample result exceed the EPA limit)
station_summary["exceedance_rate"] = round(((station_summary["compliance_failures"]/station_summary["total_samples"])*100), 2)
station_summary["severity_ratio"] = round((station_summary["peak_result"]/station_summary["mcl_value"]), 2)

station_summary

,monitoringlocationidentifier,characteristicname,total_samples,compliance_failures,peak_result,mcl_value,exceedance_rate,severity_ratio
0,11NPSWRD_WQX-SFAN_I&M_BG2,Nitrate,13,0,1.50,10.0,0.00,0.15
1,11NPSWRD_WQX-SFAN_I&M_BG2,Turbidity,13,2,11.30,5.0,15.38,2.26
2,11NPSWRD_WQX-SFAN_I&M_BJC1,Nitrate,13,0,0.41,10.0,0.00,0.04
3,11NPSWRD_WQX-SFAN_I&M_BJC1,Turbidity,13,0,4.34,5.0,0.00,0.87
4,11NPSWRD_WQX-SFAN_I&M_CDC2,Nitrate,5,0,0.54,10.0,0.00,0.05
...,...,...,...,...,...,...,...,...
803,USGS-391448122565601,Turbidity,7,6,520.00,5.0,85.71,104.00
804,YUROKTEP_WQX-TC,Turbidity,16,8,27.56,5.0,50.00,5.51
805,YUROKTEP_WQX-TG,Turbidity,19,8,36.00,5.0,42.11,7.20
806,YUROKTEP_WQX-TR,Turbidity,24,6,43.10,5.0,25.00,8.62


In [66]:
# Merging necessary columns from the water_quality dataframe to the station summary table for later data visualization
final_export = pd.merge(
    station_summary,
    water_quality[['monitoringlocationidentifier', 'monitoringlocationname', 'latitudemeasure', 'longitudemeasure']],
    on='monitoringlocationidentifier',
    how='left'
)

# Creating a function to categorize the severity of a station's exceedance
def categorize_severity(ratio):
    if ratio > 5: return "Critical"
    elif ratio > 1: return "Exceedant"
    else: return "Compliant"

# Applying the fucntion to the table
final_export['status'] = final_export['severity_ratio'].apply(categorize_severity)

# Saving the final dataframe to export for use in Tableau
final_export.to_csv("C:/Users/Liam Dujon/Desktop/Data Science/Projects/Water Quality Project/summary_data.csv", index=False)